**ПРОВЕРКА ГИПОТЕЗ ДЛЯ УВЕЛИЧЕНИЯ ВЫРУЧКИ ИНТЕРНЕТ-МАГАЗИНА**

**Описание проекта.**

В нашем распоряжении список гипотез для увеличения выручки. Необходимо приоритизировать гипотезы, запустить A/B-тест и проанализировать результаты.

**Исходные данные :**

файл hypothesis.csv - 9 гипотез по увеличению выручки интернет-магазина с указанными параметрами Reach, Impact, Confidence, Effort.

Результаты A/B-теста описаны в файлах orders.csv и visitors.csv.

Цель исследования :
Выяснить - какая из гипотез приведет к наибольшему увеличению выручки.

**Ход исследования:**

Данные -в файлах:
/datasets/hypothesis.csv
/datasets/orders.csv
/datasets/visitors.csv

О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.
Проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки попробуем исправить самые критичные ошибки данных.

Таким образом, исследование пройдёт в несколько этапов:

1.Обзор данных.

2.Предобработка данных.

3.Приоритизация гипотез.

4.Анализ A/B-теста, принятие решения по результатам теста и объяснение его.

**ОБЗОР  И ПРЕДОБРАБОТКА ДАННЫХ**

Импортируем необходимые библиотеки, загрузим данные в переменные.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import scipy.stats as stats
import numpy as np

#столбцы и строки полностью
pd.set_option('display.max_columns', None)

# выставляем ограничение на показ знаков после запятой
pd.options.display.float_format = '{:,.3f}'.format

pd.set_option('max_colwidth', 200)

In [ ]:
hypothesis = pd.read_csv('/datasets/hypothesis.csv')
orders = pd.read_csv('/datasets/orders.csv')
visitors = pd.read_csv('/datasets/visitors.csv')

In [ ]:
hypothesis

In [ ]:
hypothesis.info()

Пропущенных значений нет, дубликатов также визуально не наблюдается, тип данных соответствующий. Необходимо только привести названия столбцов к нижнему регистру.

In [ ]:
#привели название столбцов к нижнему ругистру
hypothesis.columns = hypothesis.columns.str.lower()

In [ ]:
#проверка
hypothesis

In [ ]:
orders.head()

In [ ]:
orders.info()

In [ ]:
#проверим наличие дубликатов
orders.duplicated().sum()

Пропущенных значений нет, дубликатов нет. Необходимо изменить наименования столбцов (transactionId,visitorId) и изменить тип данных столбца date.

In [ ]:
#изменим наименования столбцов (transactionId,visitorId)
orders = orders.rename(columns = {'transactionId':'transaction_id', 'visitorId':'visitor_id'})
orders.columns

In [ ]:
#изменим тип данных столбца date.
orders['date'] = pd.to_datetime(orders['date'])


In [ ]:
#проверка
orders.info()

In [ ]:
visitors.head()

In [ ]:
visitors.info()

In [ ]:
visitors.duplicated().sum()

Пропущенных значений нет, дубликатов нет. Необходимо изменить тип данных столбца date 

In [ ]:
# изменим тип данных столбца date 
visitors['date'] = pd.to_datetime(visitors['date'])

In [ ]:
#проверка
visitors.info()

**Вывод:**

В результате предобработки данных исправлены нарушения стиля в наименовании столбцов и устранено несоотвествие типа данных в столбцах, в которых обозначена дата: с типа object на datetime. Дубликаты отсутствуют. Данные готовы для анализа.

**ПРИОРИТИЗАЦИЯ ГИПОТЕЗ.**

Примените фреймворк ICE для приоритизации гипотез.

In [ ]:
hypothesis['ice'] = (hypothesis['impact'] * hypothesis['confidence']) / hypothesis['efforts']
hypothesis[['hypothesis', 'ice']].sort_values(by='ice', ascending=False)

Наиболее перспективные гипотезы по ICE: 8,0 и 7

Примените фреймворк RICE для приоритизации гипотез

In [ ]:
hypothesis['rice'] = hypothesis['reach'] * hypothesis['impact'] * hypothesis['confidence'] / hypothesis['efforts']

hypothesis[['hypothesis', 'rice']].sort_values(by='rice', ascending=False)

Теперь более преспективные гипотезы:7 - так получилось, потому что у нее параметр reach = 10,  в то время как у других гипотез он меньше,затем идут гипотезы 2 и 0.

**Вывод:**

Показатели ICE и RICE в приоритете определяют разные гипотезы, на это повлиял показатель reach - охват пользователей, которых затронет изменение, которое планируется внести. У гипотезы 7 reach составляет 10 по 10-бальной шкале, т.е. максимум. Со 2 гипотезой, аналогично, показатель reach имеет значение 8, что также повышает приоритет данной гипотезы.
А вот у 0 гипотезы, которая состоит в тройке приоритетных по ICE и RICE показатель reach не высок (3), но зато показатели impact и confidence достаточно высокие (10 и 8 соответственно).

**АНАЛИЗ А/В - теста**



Проведем анализ полученных результатов проведенного A/B-теста: с начала рассчитаем кумулятивные метрики по имеющимся в нашем распоряжении данным, проанализируем, затем проведем А/В тестирование до и после удаления выбросов.

Предварительно проверим выборки на предмет пользователей, попавших в обе группы:

In [ ]:
#выборка пользователей, попавших в обе группы

duplicated_users = orders.groupby('visitor_id').agg({'group': ['nunique', 'unique']})
duplicated_users.columns = ['groups', 'group_names']
duplicated_users = duplicated_users.query('groups > 1')
display(duplicated_users.head())
len(duplicated_users)

In [ ]:
print('Общее количество пользователей до удаления "двойных" пользователей-', orders['visitor_id'].nunique())
print('количество пользователей в группе А -', orders.query('group == "A"')['visitor_id'].nunique())
print('количество пользователей в группе В -', orders.query('group == "B"')['visitor_id'].nunique())

Таких пользователей - 58, это около 10% в каждой группе. Их участие сразу в обоих группах может субъективно исказить результаты, а также повлиять на расчет выбросов. Но поскольку нам известны только id пользователей, совершивших хоть один платеж, а "неплатящие" неизвестны, то удаление таких пользователей может также исказить результаты в сторону "неплатящих". Оставим их в данных, а результаты будем анализировать предварительно разделив данные по группам А и В.

Расчет кумулятивных метрик, необходимых для анализа.

In [ ]:
#Создадим массив уникальных пар значений дат и групп теста методом drop_duplicates()
dates_groups = orders[['date', 'group']].drop_duplicates()

dates_groups

In [ ]:
# получаем агрегированные кумулятивные по дням данные о заказах 
orders_agregate = dates_groups.apply(
    lambda x: orders[np.logical_and(orders['date'] <= x['date'], orders['group'] == x['group'])]
    .agg({'date':'max', 'group': 'max', 'transaction_id':'nunique', 'visitor_id':'nunique', 'revenue':'sum'}), axis=1).sort_values(by=['date', 'group'])

orders_agregate.head()

In [ ]:
# получаем агрегированные кумулятивные по дням данные о посетителях
visitors_agregate = dates_groups.apply(
    lambda x:visitors[np.logical_and(visitors['date'] <= x['date'], visitors['group'] == x['group'])]
    .agg({'date':'max', 'group':'max', 'visitors':'sum'}), axis=1).sort_values(by=['date', 'group'])

visitors_agregate.head()

In [ ]:
# объединяем кумулятивные данные в одной таблице и присваиваем ее столбцам понятные названия

cumulative_data = orders_agregate.merge(
    visitors_agregate, 
    left_on=['date', 'group'], 
    right_on=['date', 'group']
)

cumulative_data.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']
cumulative_data.head()

1.Построим график кумулятивной выручки по группам

In [ ]:
# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе А
cumulative_revenue_a = cumulative_data[cumulative_data['group']=='A'][['date','revenue', 'orders']]

# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе B
cumulative_revenue_b = cumulative_data[cumulative_data['group']=='B'][['date','revenue', 'orders']]

# Строим график выручки группы А
plt.plot(cumulative_revenue_a['date'], cumulative_revenue_a['revenue'], label='A')

# Строим график выручки группы B
plt.plot(cumulative_revenue_b['date'], cumulative_revenue_b['revenue'], label='B')


plt.title('График кумулятивной выручки по группам А и В')
plt.ylabel("Выручка, млн.")
plt.xticks(rotation=45)
plt.legend()
plt.show()



**Вывод:**

На графике кумулятивной выручки заметно, что кривая выручки группы B выше, чем группы А. Кроме того, где-то в период 18-19 августа произошел ее резкий скачок, в результате которого разрыв между группами стал еще более значительным, далее обе линии идут параллельно.

Вероятнее всего такое резкое изменение произошло из-за крупного заказа (заказов).

2.Построим график кумулятивного среднего чека по группам.
Построим график, аналогичный предыдущему, только вместо выручки найдем средний чек (выручка поделенная на заказы)

In [ ]:

plt.plot(cumulative_revenue_a['date'], cumulative_revenue_a['revenue']/cumulative_revenue_a['orders'], label='A')
plt.plot(cumulative_revenue_b['date'], cumulative_revenue_b['revenue']/cumulative_revenue_b['orders'], label='B')
plt.title('Кумулятивный средний чек по группам\n')
plt.xlabel('Дата')
plt.ylabel('Средний чек')
plt.xticks(rotation=45)
plt.legend() 
plt.show()

Мы видим, что кривая группы В в период с 11 по 15 августа падает ниже уровня группы А, но затем резко растет, что еще раз подтверждает, что в группе В в период 18-19 августа была крупная покупка, которая увеличила размер кумулятивного среднего чека.

3.Построим график относительного изменения кумулятивного среднего чека группы B к группе A.

In [ ]:
# собираем данные в одном датафрейме
merged_cumulative_revenue = cumulative_revenue_a.merge(
    cumulative_revenue_b, 
    left_on='date', 
    right_on='date', 
    how='left', 
    suffixes=['_a', '_b']
)

# cтроим отношение средних чеков
plt.plot(merged_cumulative_revenue['date'], 
         (
             merged_cumulative_revenue['revenue_b']/merged_cumulative_revenue['orders_b']
         )/
         (
             merged_cumulative_revenue['revenue_a']/merged_cumulative_revenue['orders_a']
         )-1
        )

# добавляем ось X
plt.axhline(y=0, color='black', linestyle='--')
plt.title('График относительного изменения кумулятивного среднего чека группы B к группе A')
plt.xticks(rotation=45)
plt.show()

График резко скачет в нескольких точках, следовательно, имеют место какие-то выбросы и крупные заказы.

4. Построим график кумулятивного среднего количества заказов на посетителя по группам

In [ ]:
cumulative_data['ord'] = cumulative_data['orders']/cumulative_data['visitors']

cumulative_data_a = cumulative_data[cumulative_data['group']=='A']
cumulative_data_b = cumulative_data[cumulative_data['group']=='B']

plt.plot(cumulative_data_a['date'], cumulative_data_a['ord'], label='A')
plt.plot(cumulative_data_b['date'], cumulative_data_b['ord'], label='B')
plt.legend()
plt.title('График кумулятивного среднего количества заказов на посетителя по группам')
plt.xlabel('Дата')
plt.ylabel('Количество заказов')
plt.xticks(rotation=45)
plt.show()

Графики ассиметричны, первоначально преобладала группа А, но затем группа В вырвалась вперёд , а среднее количество заказов группы А просело.

5. Построим график относительного изменения кумулятивного среднего количества заказов на посетителя группы B к группе A

In [ ]:
merged_сumulative_ord = (
    cumulative_data_a[['date','ord']]
    .merge(
        cumulative_data_b[['date','ord']], 
        left_on='date', 
        right_on='date', 
        how='left', 
        suffixes=['_a', '_b']
    )
)
plt.plot(
    merged_сumulative_ord ['date'], 
    merged_сumulative_ord ['ord_b']/
    merged_сumulative_ord ['ord_a']-1
)

# добавляем ось X
plt.axhline(y=0, color='black', linestyle='--')
plt.axhline(y=0.15, color='grey', linestyle='--')
plt.title('График относительного изменения кумулятивного среднего количества заказов на посетителя группы B к группе A')
plt.xticks(rotation=30)
plt.show()

В начале теста группа В проигрывала группе А, но затем стремительно начала расти и вырвалась вперед, далее начался медленный спад. Теперь  группа В снова растет.

Необходимо проанализировать данные после очистки от выбросов еще раз.

Изучим гистограмму распределения количества заказов: 

In [ ]:
orders_by_users = (
    orders.groupby('visitor_id', as_index=False)
    .agg({'transaction_id': 'nunique'})
)
orders_by_users.columns =  ['visitor_id', 'orders']

#display(orders_by_users['orders'].describe())

# строим гистограмму
plt.hist(orders_by_users['orders'], alpha=0.5) 
plt.title('Распределение количества заказов по пользователям')
plt.xlabel('Количество заказов')
plt.ylabel('Количество пользователей')
plt.show()

Большинство пользователей оформляли заказ один раз, но есть и те, кто успел сделать за месяц одиннадцать заказов.

6. Построим точечную диаграмму числа заказов на одного пользователя:

In [ ]:
x_values = pd.Series(range(0,len(orders_by_users)))

# строим точечную диаграмму
plt.scatter(x_values, orders_by_users['orders'], color='blue', alpha=0.5)
plt.title('Распределение количества заказов по пользователям')
plt.show()

Много пользователей с 2-3 заказами. Их точная доля не ясна, поэтому сложно понять, можно ли считать их выбросами или нет.


7. Посчитаем 95-й и 99-й перцентили количества заказов на пользователя и выберем границу для определения аномальных пользователей.

In [ ]:
np.percentile(orders_by_users['orders'], [95, 99])

То есть, примерно 5% пользователей сделали больше 2 заказов, и 1% пользователей сделали больще 4 заказов.
Разумно выбрать 2 заказа на одного пользователя за нижнюю границу числа заказов, и отсеять аномальных пользователей по ней.

Изучим гистограмму стоимости заказов.

In [ ]:
plt.hist(orders['revenue'], alpha=0.5) 
plt.title('Распределение стоимости заказов по пользователям')
plt.xlabel('Стоимость заказов')
plt.ylabel('Количество пользователей')
plt.show()

По гистограмме видно, что большинство заказом имеют низкую стоимость.

8. Постройте точечный график стоимостей заказов

In [ ]:
x_values = pd.Series(range(0,len(orders)))
plt.figure(figsize=(10,5))
plt.ylabel('Стоимость заказа')
plt.xlabel('Количество')
plt.title('Точечная диаграмма стоимости заказов')
plt.scatter(x_values, orders['revenue']);

На точечной диаграмме  мы уже видим явняе аномалии - один заказ на сумму более 1.2 млн, является ли заказ на сумму около 200 тыс. аномалией можно понять, опеределив 95 и 99 перцентили

In [ ]:
np.percentile(orders['revenue'], [95,99])

Не более 5% заказов дороже 28000 рублей и не более 1% дороже 58233.2 рублей. Определим нижнюю границу стоимости заказов равной 28000р. Предполагаемые аномалии действительно являются аномалиями.

**Анализ A/B-теста**

Ранее мы рассмотрели результаты A/B-теста визуально и выяснили, что в данных, скорее всего, есть выбросы. Увидели выбросы и нашли границу для их определения.
Посчитаем статистическую значимость различий в среднем числе заказов на пользователя и среднем чеке между группами по «сырым» данным — без удаления аномальных пользователей.



Создадим переменные orders_by_users_a и orders_by_users_b со столбцами ['visitor_id', 'orders']. В них для пользователей, которые заказывали хотя бы 1 раз, укажем число совершённых заказов

In [ ]:
orders_by_users_a = (orders[orders['group']=='A']
                     .groupby('visitor_id', as_index=False)
                     .agg({'transaction_id':pd.Series.nunique})
                    )
orders_by_users_a.columns = ['visitor_id', 'orders']


In [ ]:
orders_by_users_b = (orders[orders['group']=='B']
                     .groupby('visitor_id', as_index=False)
                     .agg({'transaction_id':pd.Series.nunique})
                    )
orders_by_users_b.columns = ['visitor_id', 'orders']


Объявим переменные sample_a и sample_b, в которых пользователям из разных групп будет соответствовать количество заказов. Тем, кто ничего не заказал, будут соответствовать нули. Это нужно, чтобы подготовить выборки к проверке критерием Манна-Уитни.
Объединим последовательности функцией pd.concat(). 

In [ ]:
sample_a = pd.concat(
    [orders_by_users_a['orders'],
     pd.Series(0, index=np.arange(visitors[visitors['group'] == 'A']['visitors'].sum() - len(orders_by_users_a['orders'])), name='orders',)
     ,], axis=0,
)

sample_b = pd.concat(
    [orders_by_users_b['orders'],
     pd.Series(0, index=np.arange(visitors[visitors['group'] == 'A']['visitors'].sum() - len(orders_by_users_b['orders'])), name='orders',)
     ,], axis=0,
)

10. Посчитаем статистическую значимость различий в среднем числе заказов на пользователя и среднем чеке между группами по «сырым» данным — без удаления аномальных пользователей.

Сформулируем гипотезы. 

Ho: различий в среднем количестве заказов между группами нет. 

H1: различия в среднем между группами есть.

Для тестирования воспользуемся критерием Манна-Уитни, уровнем статистической значимости примем значение 0,05

In [ ]:
#расчет p-value критерия Манна-Уитни
print("p-value {0:.3f}".format(stats.mannwhitneyu(sample_a, sample_b)[1]))


In [ ]:
print("прирост метрики {0:.3f}".format(sample_b.mean() / sample_a.mean() - 1))

P-value = 0.011 меньше 0.05, значит нулевую гипотезу о том, что статистически значимых различий в среднем числе заказов между группами нет, отвергаем. Относительный прирост среднего группы B к группе А равен 14.9 %.


11. Теперь проверим статистическую значимость различий в среднем чеке между группами.

Сформулируем гипотезы:

H₀ - статистически значимых различий в среднем чеке между группами нет

H1: статистически значимые различия в среднем чеке между группами есть

уровень статистической значимости примем равным 0.05

In [ ]:
#тест Манна-Уитни
print('p-value {0:.3f}'.format(stats.mannwhitneyu(orders[orders['group']=='A']['revenue'], orders[orders['group']=='B']['revenue'], True)[1]))

#относительные различия суммы среднего чека
print('прирост метрики {0:.3f}'.format(orders[orders['group']=='B']['revenue'].mean()/orders[orders['group']=='A']['revenue'].mean()-1)) 


По "сырым" данным значение p-value = 0.729, что значительно больше уровня статистической значимости, следовательно гипотезу о том, что статически значимых различий в сумме среднего чека между группами нет, не отвергаем. Относительная разница суммы среднего чека между группами составила 25.9%- на столько сумма среднего чека группы В больше суммы среднего чека группы А

12.Посчитаем статистическую значимость различий в среднем количестве заказов на посетителя между группами по «очищенным» данным.

Найденные 95-й и 99-й перцентили средних чеков равны 28000 и 58233.2 рублям. А 95-й и 99-й перцентили количества заказов на одного пользователя равны 2 и 4 заказа на пользователя.
Примем за аномальных пользователей тех, кто совершил от 3 заказов или совершил заказ дороже 28 000 рублей. Так мы уберём от 1% до 5% пользователей с наибольшим числом заказов и 5% пользователей с дорогими заказами. Сделаем срезы пользователей с числом заказов больше 2 — users_with_many_orders и пользователей, совершивших заказы дороже 28 000 — users_with_expensive_orders. Объединим их в таблице abnormal_users.

In [ ]:
users_with_many_orders = pd.concat(
    [
        orders_by_users_a[orders_by_users_a['orders'] > 2]['visitor_id'],
        orders_by_users_b[orders_by_users_b['orders'] > 2]['visitor_id'],
    ],
    axis=0,)


In [ ]:
users_with_expensive_orders = orders[orders['revenue'] > 28000]['visitor_id']


In [ ]:
abnormal_users = (
    pd.concat([users_with_many_orders, users_with_expensive_orders], axis=0)
    .drop_duplicates()
    .sort_values()
)
abnormal_users

Получилось 74 аномальных пользователя,
узнаем как их действия повлияли на результаты теста. 

Посчитаем статистическую значимость различий в среднем количестве заказов на посетителя между группами по «очищенным» данным.

In [ ]:
sample_a_filtered = pd.concat(
    [orders_by_users_a[np.logical_not(orders_by_users_a['visitor_id'].isin(abnormal_users))]['orders'],
        pd.Series(0, index=np.arange(visitors[visitors['group'] == 'A']['visitors'].sum() - len(orders_by_users_a['orders'])),
            name='orders'),], axis=0)


In [ ]:
sample_b_filtered = pd.concat(
    [orders_by_users_b[np.logical_not(orders_by_users_b['visitor_id'].isin(abnormal_users))]['orders'],
        pd.Series(0, index=np.arange(visitors[visitors['group'] == 'A']['visitors'].sum() - len(orders_by_users_b['orders'])),
            name='orders'),], axis=0)


Применим статистический критерий Манна-Уитни к полученным выборкам

Сформулируем гипотезы:

H₀ - статистически значимых различий в среднем количестве заказов между группами нет

H1 - статистически значимые различия в среднем количестве заказов между группами есть

уровень статистической значимости примем равным 0.05

In [ ]:
print('p-value {0:.3f}'.format(stats.mannwhitneyu(sample_a_filtered, sample_b_filtered)[1]))
print('прирост метрики {0:.3f}'.format(sample_b_filtered.mean()/sample_a_filtered.mean()-1)) 

Результаты по среднему количеству заказов практически не изменились (p-value = 0.011 по "сырым" данным),p-value = 0.013 меньше 0.05, значит нулевую гипотезу о том, что статистически значимых различий в среднем количестве заказов между группами нет, отвергаем.
Относительный прирост среднего группы B к группе А равен 17.3 % (по "сырым" данным прирост составлял 14.9 %).

13. Посчитаем статистическую значимость различий в среднем чеке заказа между группами по «очищенным» данным

сформулируем гипотезы: 

Hо - статистически значимых различий в среднем чеке между группами нет 

H₁- статистически значимые различия в среднем чеке между группами есть

уровень статистической значимости примем равным 0.05

In [ ]:
print('p-value {0:.3f}'.format(stats.mannwhitneyu(
    orders[np.logical_and(
        orders['group'] == 'A',
        np.logical_not(orders['visitor_id'].isin(abnormal_users)),
    )]['revenue'],
    orders[np.logical_and(
        orders['group'] == 'B',
        np.logical_not(orders['visitor_id'].isin(abnormal_users)),
    )]['revenue'],)[1]))

In [ ]:
print("прирост метрики {0:.3f}".format(orders[np.logical_and(
        orders['group'] == 'B',
        np.logical_not(orders['visitor_id'].isin(abnormal_users)),
        )]['revenue'].mean()
    / orders[np.logical_and(
        orders['group'] == 'A',
        np.logical_not(orders['visitor_id'].isin(abnormal_users)),
        )]['revenue'].mean()
        - 1))

Результаты p-value по среднему чеку также практически не изменились, нулевую гипотезу о том, что статистически значимых различий в среднем чеке между группами нет, не отвергаем. НО, средний чек группы B стал ниже среднего чека группы A.

14. Сформулируем выводы и имеющиеся факты:

- статистически значимые различия в среднем количестве заказов между группамии по «сырым» и по данным после фильтрации аномалий присутствуют,показатели группы В выше, чем А, на 15-17%;
- нет статистически значимого различия по среднему чеку между группами ни по «сырым», ни по данным после фильтрации аномалий,при этом средний чек группы В выше (на "очищенных" данных - на ~2%);


Если целью тестирования было установить -изменилась ли сумма среднего чека, то она осталась прежней, мы видим лишь незначительные изменения. Если же целью тестирования являлось изменение количества заказов, то результаты тестирования группы В говорят о том, что ее результаты отличается от группы А на 15-17%.

Тест остановить, зафиксировать отсутствие различий между группами по сумме среднего чека и победу группы В в части увеличения количества заказов.

**ИТОГ:**

На основании входных данных, предоставленных интернет-магазином был проведено исследование и вынесены рекомендации, изложенные ниже.

1. В части приоритизации гипотез из списка, предоставленных отделом Маркетинга следует в первую очередь обратить внимание на гипотезы:
- "Запустить акцию, дающую скидку на товар в день рождения",
- "Добавить два новых канала привлечения трафика, что позволит привлекать на 30% больше пользователей",
- "Добавить форму подписки на все основные страницы, чтобы собрать базу клиентов для email-рассылок";

Если приоритизации гипотез должна включать в себя и охват пользователей интернет-магазина, то места необходимо распределить таким образом:

- "Добавить форму подписки на все основные страницы, чтобы собрать базу клиентов для email-рассылок",
- "Добавить блоки рекомендаций товаров на сайт интернет магазина, чтобы повысить конверсию и средний чек заказа",
- "Добавить два новых канала привлечения трафика, что позволит привлекать на 30% больше пользователей".

2. В части анализа А/В теста:

- Есть статистически значимое различие по среднему количеству заказов между группами как по «сырым», так и по данным после фильтрации аномалий. Показатели группы В выше, чем А, на 15-17%;

- Нет статистически значимого различия по среднему чеку между группами ни по «сырым», ни по данным после фильтрации аномалий. При этом средний чек группы В выше (на "очищенных" данных - на ~2%);


На основании вышеизложенного рекомендуем остановить тест, зафиксировав победу группы B (в части увеличения количества заказов).
